# Python Load Balancer with Interactive Dashboard

This notebook implements a TCP load balancer with a real-time monitoring dashboard. The load balancer distributes incoming connections across multiple backend servers using a round-robin algorithm.

## Features
- TCP load balancing with configurable backends
- Real-time connection monitoring
- Interactive dashboard with statistics and visualizations
- Connection management interface

## Install Required Packages

First, let's make sure we have all the required packages installed.

In [ ]:
# Uncomment and run this cell to install required packages
# !pip install ipywidgets pandas plotly

## Import Necessary Libraries

In [ ]:
# Import standard libraries
import socket
import threading
import time
import random
import logging
import uuid
from datetime import datetime, timedelta
import queue

# Import third-party libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import plotly.graph_objs as go

# Import our load balancer modules
from loadbalancer.core import LBManager
from loadbalancer.stats import StatsCollector
from loadbalancer.ui import LoadBalancerUI

## Initialize the Load Balancer

In [ ]:
# Create instances of our classes
lb_manager = LBManager()
stats_collector = StatsCollector(lb_manager)
lb_ui = LoadBalancerUI(lb_manager, stats_collector)

# Display the UI
lb_ui.display()

## Using the Load Balancer

### Basic Usage
1. Set the port on which the load balancer should listen (default: 8080)
2. Enter the backend server addresses in the format `host:port`, one per line
3. Click the "Start" button to start the load balancer
4. The dashboard will show real-time statistics and active connections
5. Click the "Stop" button to stop the load balancer

### Testing the Load Balancer
You can test the load balancer by setting up multiple server instances and then connecting clients to the load balancer port. For example, you could use simple Python socket servers as backends, and then use tools like `telnet` or `netcat` to connect to the load balancer.

Example test backend server (run in separate terminals):
```python
import socket
import threading

def handle_client(conn, addr, server_id):
    print(f"Server {server_id}: New connection from {addr}")
    conn.send(f"Hello from backend server {server_id}\n".encode())
    while True:
        try:
            data = conn.recv(1024)
            if not data:
                break
            print(f"Server {server_id} received: {data.decode().strip()}")
            conn.send(f"Server {server_id} echo: {data.decode()}".encode())
        except:
            break
    conn.close()
    print(f"Server {server_id}: Connection closed from {addr}")

def start_server(port, server_id):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind(('0.0.0.0', port))
    server.listen(5)
    print(f"Server {server_id} listening on port {port}")
    while True:
        conn, addr = server.accept()
        client_thread = threading.Thread(target=handle_client, args=(conn, addr, server_id))
        client_thread.daemon = True
        client_thread.start()

# Start server
server_id = 1  # Change for each server
port = 8081  # Change for each server
start_server(port, server_id)
```

## Example: Create Simple Backend Servers

The following cell creates two simple backend servers for testing purposes.

In [ ]:
def create_test_servers():
    """
    Create simple backend servers for testing.
    Returns the server threads and a list of server ports.
    """
    def handle_client(conn, addr, server_id):
        print(f"Server {server_id}: New connection from {addr}")
        conn.send(f"Hello from backend server {server_id}\n".encode())
        while True:
            try:
                data = conn.recv(1024)
                if not data:
                    break
                print(f"Server {server_id} received: {data.decode().strip()}")
                conn.send(f"Server {server_id} echo: {data.decode()}".encode())
            except:
                break
        conn.close()
        print(f"Server {server_id}: Connection closed from {addr}")

    def server_thread(port, server_id):
        server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        server.bind(('0.0.0.0', port))
        server.listen(5)
        print(f"Server {server_id} listening on port {port}")
        
        while True:
            try:
                conn, addr = server.accept()
                client_thread = threading.Thread(target=handle_client, args=(conn, addr, server_id))
                client_thread.daemon = True
                client_thread.start()
            except:
                break
                
        server.close()
        print(f"Server {server_id} stopped")
    
    # Create and start two test servers
    ports = [8081, 8082]
    threads = []
    
    for i, port in enumerate(ports):
        thread = threading.Thread(target=server_thread, args=(port, i+1))
        thread.daemon = True
        thread.start()
        threads.append(thread)
    
    return threads, ports

# Start test servers
server_threads, server_ports = create_test_servers()
print(f"Started {len(server_ports)} test servers on ports {', '.join(map(str, server_ports))}")

## Example: Test Client Connection

The following function creates a test client that connects to the load balancer.

In [ ]:
def test_client(lb_port, message="Hello from test client", num_messages=5):
    """
    Create a test client that connects to the load balancer and sends messages.
    """
    try:
        # Connect to the load balancer
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client.connect(('localhost', lb_port))
        
        # Receive the welcome message
        welcome = client.recv(1024).decode()
        print(f"Received: {welcome}")
        
        # Send some test messages
        for i in range(num_messages):
            msg = f"{message} #{i+1}"
            print(f"Sending: {msg}")
            client.send(msg.encode())
            
            # Receive the response
            response = client.recv(1024).decode()
            print(f"Received: {response}")
            
            # Add a small delay between messages
            time.sleep(0.5)
        
        # Close the connection
        client.close()
        print("Client connection closed")
        
    except Exception as e:
        print(f"Client error: {e}")

# Example usage (uncomment to run):
# test_client(8080)

## Load Testing

This function creates multiple client connections to test the load balancer under load.

In [ ]:
def load_test(lb_port, num_clients=10, messages_per_client=3):
    """
    Perform a load test by creating multiple client connections.
    """
    def client_thread(client_id):
        try:
            test_client(lb_port, f"Client {client_id} message", messages_per_client)
        except Exception as e:
            print(f"Client {client_id} error: {e}")
    
    # Create and start client threads
    threads = []
    for i in range(num_clients):
        thread = threading.Thread(target=client_thread, args=(i+1,))
        thread.daemon = True
        threads.append(thread)
    
    # Start all client threads
    print(f"Starting {num_clients} test clients connecting to port {lb_port}...")
    for thread in threads:
        thread.start()
        # Add a small delay between client starts to better visualize connections
        time.sleep(0.2)
    
    # Wait for all clients to finish
    for thread in threads:
        thread.join()
    
    print("Load test completed")

# Example usage (uncomment to run):
# load_test(8080, num_clients=5)

## Cleanup

Make sure to stop the load balancer and clean up resources when you're done.

In [ ]:
def cleanup():
    """Stop the load balancer and clean up resources."""
    # Stop the load balancer if it's running
    if lb_manager.is_running():
        lb_manager.stop_listener()
        print("Load balancer stopped")
    
    # Stop the stats collector
    stats_collector.stop()
    
    # Shutdown the UI
    lb_ui.shutdown()
    
    print("Cleanup completed")

# Register cleanup to run when the notebook is closed
import atexit
atexit.register(cleanup)

## Conclusions and Improvements

This interactive load balancer demonstrates the following capabilities:

1. **TCP Load Balancing**: Distributes connections across multiple backend servers
2. **Real-time Monitoring**: Shows active connections and statistics
3. **Interactive UI**: Control the load balancer through a user-friendly interface
4. **Data Visualization**: Graphs showing connection counts and throughput

### Possible Improvements

1. **Additional Load Balancing Algorithms**: Implement weighted round-robin, least connections, IP hash, etc.
2. **SSL/TLS Support**: Add support for secure connections
3. **Health Checks**: Periodically check backend server health and remove unhealthy servers
4. **Connection Limiting**: Implement connection limits to prevent overload
5. **Sticky Sessions**: Enable session persistence for certain types of applications
6. **Export/Import Configuration**: Save and load load balancer configuration
7. **REST API**: Add a REST API for programmatic control
8. **User Authentication**: Add user authentication for the admin interface
9. **Docker Integration**: Package as a containerized application
10. **Improved Logging**: Enhanced logging with different log levels and output formats